In [1]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader


## dataset

In [3]:
class GTZAN_Dataset(Dataset):
    def __init__(self, dataset, dir, transform=None, target_transform=None):
        self.labels = pd.read_csv(dataset, dtype={'file':str, 'category':int})
        self.dir = dir
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        filepath = os.path.join(self.dir, self.labels.iloc[idx,0])
        spec = torch.load(filepath)
        label = self.labels.iloc[idx,1]
        
        if self.transform:
            spec = self.transform(spec)
        if self.target_transform:
            label = self.target_transform(label)
        
        return spec, label

## data loader

In [9]:
root_path = '/home/christian/Documents/sound_classifier'

gtzan = GTZAN_Dataset('dataset_files.csv', root_path)
train_dataloader = DataLoader(gtzan, batch_size=64, shuffle=True)
test_dataloader = DataLoader(gtzan, batch_size=64, shuffle=True)

train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([64, 1, 401, 166])
Labels batch shape: torch.Size([64])


## model

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            # dimensions of the tensor created by torchaudio: 1*401*166
            nn.Linear(401*166, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=66566, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
# tensor consits of complex numbers, has to be transformed to real numbers
X = train_features.real

logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9, 1, 1, 1, 2, 0, 2, 1, 1, 2, 9, 1, 2, 5, 9, 9, 1, 1, 1, 9, 0, 1, 1, 9,
        1, 5, 1, 0, 1, 5, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 9, 3, 2, 3, 0, 1, 0, 1,
        1, 0, 1, 1, 1, 3, 0, 9, 1, 1, 9, 0, 9, 1, 0, 1])


## training

In [13]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.real)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.real)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304509  [    0/ 9991]
loss: 2.277025  [ 6400/ 9991]
Test Error: 
 Accuracy: 25.7%, Avg loss: 2.183204 

Epoch 2
-------------------------------
loss: 2.138200  [    0/ 9991]
loss: 2.188426  [ 6400/ 9991]
Test Error: 
 Accuracy: 35.1%, Avg loss: 2.083098 

Epoch 3
-------------------------------
loss: 2.124776  [    0/ 9991]
loss: 2.134519  [ 6400/ 9991]
Test Error: 
 Accuracy: 42.1%, Avg loss: 1.994195 

Epoch 4
-------------------------------
loss: 1.993377  [    0/ 9991]
loss: 2.070077  [ 6400/ 9991]
Test Error: 
 Accuracy: 49.9%, Avg loss: 1.910784 

Epoch 5
-------------------------------
loss: 1.937874  [    0/ 9991]
loss: 1.880027  [ 6400/ 9991]
Test Error: 
 Accuracy: 58.2%, Avg loss: 1.829950 

Done!


In [17]:
torch.save(model, 'sound_classifier_simple_model.pth')